In [1]:
#SORT lists by topology and update overleaf
#keep fwd/bwd l-r labels on topology.

In [2]:
import sys
import os

sys.path.append(os.getcwd()[:-9])
from src.ops.operator import Operator
from src.ops.elemental import ElementalOperator
from src.ops.quarks import Quark
from src.ops.indexed import IndexedObject
from src.wick.contract import contract

In [3]:
q0=Quark(True,'u','s_0','c_0','t_3','x_3')
g0=IndexedObject('\\gamma_5',['s_0','s_1'])
q1=Quark(False,'d','s_1','c_0','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g0],[q0,q1])])

#TODO add charge factors qu and qd

elementals=[]
for pi2 in [['u',+2/3],['d',-1/3]]:                              # uu + dd
    for pi1 in [['u',+2/3],['d',-1/3]]:                          # uu + dd 
        for dirpi2 in [-1,+1]:                               # eq 17 of notes
            for dirpi1 in [-1,+1]:
                q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                if(dirpi1==-1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
                    g1=IndexedObject('(1-\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2+a\hat{\mu}')
                elif(dirpi1==+1):
                    q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2+a\hat{\mu}')
                    g1=IndexedObject('(1+\\gamma_{\mu})',['s_2','s_3'])
                    g2=IndexedObject('U_{\mu}^{\dagger}(x_2)',['c_1','c_2'])
                    q3=Quark(False,pi2[0],'s_3','c_2','t_2','x_2')
    
                if(dirpi2==-1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1')
                    g3=IndexedObject('(1-\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1+a\hat{\\nu}')
                    
                elif(dirpi2==+1):
                    q4=Quark(True,pi1[0],'s_4','c_3','t_1','x_1+a\hat{\\nu}')
                    g3=IndexedObject('(1+\\gamma_{\\nu})',['s_4','s_5'])
                    g4=IndexedObject('U_{\\nu}^{\dagger}(x_1)',['c_3','c_4'])
                    q5=Quark(False,pi1[0],'s_5','c_4','t_1','x_1')
        
                # for printing out the insertion cleanly, just comment out this pion
                # and uncomment out the block about insertion.
                
                q6=Quark(True,'d','s_6','c_3','t_0','x_0')
                g5=IndexedObject('\\gamma_5',['s_6','s_7'])
                q7=Quark(False,'u','s_7','c_3','t_0','x_0')
                coef=pi1[1]*pi2[1]*dirpi1*dirpi2
                elementals.append(ElementalOperator(-1*coef,[g1,g2,g3,g4,g5],[q2,q3,q4,q5,q6,q7]))
                #elementals.append(ElementalOperator(coef,[g1,g2,g3,g4],[q2,q3,q4,q5]))
        
insertion=Operator(elementals)
#s=''
#for e in insertion.elementals:
#    s+=str(e)+'\\\\'
#print(s.replace('*',''))

creation=Operator(elementals)

In [4]:
res=contract(annihilate,creation)
print(len(res))
res[0].coef

80


-0.4444444444444444

In [5]:
import copy

def latexStr(gamma):
    tmp=gamma.name.split('_')
    if(tmp[0]=='(1-\\gamma)') or (tmp[0]=='(1+\\gamma)'):
        return tmp[0] + '_{' + tmp[1][:-1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    elif tmp[0][0]=='U':
        return '('+gamma.name+')'+ '_{{' + gamma.indices[0] + gamma.indices[1] + '}}'
    elif tmp[0]=='\\gamma':
        return tmp[0] + '_{' + tmp[1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    else:
        return str(gamma)
  
def platexStr(p):
    return p.name + '(' + p.xi + ',' + p.ti + '\mid ' + p.xf + ',' + p.tf + ')' + '_{\\substack{' + p.left_indices.s + '\\\\' + p.left_indices.c + '}' +  '\\substack{' + p.right_indices.s + '\\\\' + p.right_indices.c + '}}'
    
total=copy.deepcopy(res)

print(len(total))

for d in total:
    for p in d.props:
        p.name = 'S'
        #p.name = 'S^'+p.name.split('^')[1]
        
for i in range(len(total)-1,-1,-1):
    for j in range(len(total)-1,-1,-1):
        if(i!=j):
            if((total[i].commuting==total[j].commuting) and (total[i].props==total[j].props)):
                total[i].coef+=total[j].coef
                total.pop(j)
                break
for i in range(len(total)-1,-1,-1):
    if(total[i].coef==0):
        total.pop(i)
    else:
        total[i].coef*=9
        
print(len(total))

80
48


In [6]:
def print_diagrams(total_diags):
    print('\\beqs')
    for i,d in enumerate(total_diags):
        #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}='+str(d.coef) + ' '
        for p in d.props:
            tmp+=platexStr(p)
        for c in d.commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')
    print('\\eeqs')
    

In [7]:
#print_diagrams(total)

In [8]:
def diagram_time_order(total_diags,log=False):
    diagrams = {}

    for i,d in enumerate(total):
        top = []
        d_str='d_' + str(i) + ':  '
        for p in d.props:
            d_str+=p.tf + '->' + p.ti + '   '
            top.append([p.tf.split('_')[1],p.ti.split('_')[1]])
        diagrams[i]=top
        if(log):
            print(d_str)
    return diagrams

refTopologies={}
refTopologies['A']=[['1','0'],['0','2'],['2','3'],['3','1']]
refTopologies['A-bwd']=[['0','1'],['1','3'],['3','2'],['2','0']]

# t3 t2 t1 t0

refTopologies['B']=[['1','0'],['0','3'],['3','2'],['2','1']]
refTopologies['B-bwd']=[['0','1'],['1','2'],['2','3'],['3','0']]

refTopologies['C']=[['2','0'],['0','3'],['3','1'],['1','2']]
refTopologies['C-bwd']=[['0','2'],['2','1'],['1','3'],['3','0']]

refTopologies['D']=[['0','3'],['3','0'],['2','1'],['1','2']]
refTopologies['El']=[['2','2'],['3','1'],['1','0'],['0','3']]
refTopologies['El-bwd']=[['2','2'],['1','3'],['3','0'],['0','1']]
refTopologies['Er']=[['1','1'],['3','2'],['2','0'],['0','3']]
refTopologies['Er-bwd']=[['1','1'],['2','3'],['3','0'],['0','2']]
refTopologies['F']=[['0','3'],['3','0'],['1','1'],['2','2']]

from src.wick.utilities import permutations

def swap(list, i, j):
    list[i], list[j] = list[j], list[i]
    return list

def cyclic_permutations(list):
    return [[list[i-j] for i in range(len(list))] for j in range(len(list))]

#I kind of forget what this does...
def connect_times(topList):
    for idx,d in topList.items():
        for p in range(len(d)):
            if d[p][1] != d[(p+1)%len(d)][0]:
                for pp in range(p,len(d)):
                    if d[p][1] == d[pp][0]:
                        swap(d,(p+1)%len(d),pp)

def name_topologies(topList):
    for idx,d in topList.items():
        for label,top in refTopologies.items():
            if d in permutations(top):
                topList[idx]=label


In [17]:
topList=diagram_time_order(total)

connect_times(topList)

name_topologies(topList)

{0: [['3', '1'], ['2', '0'], ['1', '2'], ['0', '3']], 1: [['3', '2'], ['2', '1'], ['1', '0'], ['0', '3']], 2: [['3', '1'], ['2', '0'], ['1', '2'], ['0', '3']], 3: [['3', '2'], ['2', '1'], ['1', '0'], ['0', '3']], 4: [['3', '1'], ['2', '0'], ['1', '2'], ['0', '3']], 5: [['3', '2'], ['2', '1'], ['1', '0'], ['0', '3']], 6: [['3', '1'], ['2', '0'], ['1', '2'], ['0', '3']], 7: [['3', '2'], ['2', '1'], ['1', '0'], ['0', '3']], 8: [['3', '2'], ['2', '0'], ['1', '3'], ['0', '1']], 9: [['3', '2'], ['2', '0'], ['1', '1'], ['0', '3']], 10: [['3', '2'], ['2', '0'], ['1', '3'], ['0', '1']], 11: [['3', '2'], ['2', '0'], ['1', '1'], ['0', '3']], 12: [['3', '2'], ['2', '0'], ['1', '3'], ['0', '1']], 13: [['3', '2'], ['2', '0'], ['1', '1'], ['0', '3']], 14: [['3', '2'], ['2', '0'], ['1', '3'], ['0', '1']], 15: [['3', '2'], ['2', '0'], ['1', '1'], ['0', '3']], 16: [['3', '1'], ['2', '3'], ['1', '0'], ['0', '2']], 17: [['3', '1'], ['2', '2'], ['1', '0'], ['0', '3']], 18: [['3', '1'], ['2', '3'], ['1', '0

In [10]:
print(topList)

{0: 'C', 1: 'B', 2: 'C', 3: 'B', 4: 'C', 5: 'B', 6: 'C', 7: 'B', 8: 'A-bwd', 9: 'Er', 10: 'A-bwd', 11: 'Er', 12: 'A-bwd', 13: 'Er', 14: 'A-bwd', 15: 'Er', 16: 'A', 17: 'El', 18: 'A', 19: 'El', 20: 'A', 21: 'El', 22: 'A', 23: 'El', 24: 'El-bwd', 25: 'B-bwd', 26: 'Er-bwd', 27: 'C-bwd', 28: 'D', 29: 'F', 30: 'El-bwd', 31: 'B-bwd', 32: 'Er-bwd', 33: 'C-bwd', 34: 'D', 35: 'F', 36: 'El-bwd', 37: 'B-bwd', 38: 'Er-bwd', 39: 'C-bwd', 40: 'D', 41: 'F', 42: 'El-bwd', 43: 'B-bwd', 44: 'Er-bwd', 45: 'C-bwd', 46: 'D', 47: 'F'}


In [11]:
x=copy.deepcopy(topList)
topSorted={k: v for k, v in sorted(x.items(), key=lambda item: item[1])}

In [14]:
def print_diagrams_labelled(diags,sortd):
    print('\\beqs')
    newI=0
    for i,label in sortd.items():
        #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}^{\\text{('+label+')}}='+str(diags[i].coef) + ' '
        for p in diags[i].props:
            tmp+=platexStr(p)
        for c in diags[i].commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')
        newI+=1
    print('\\eeqs')

In [15]:
print_diagrams_labelled(total,topSorted)

\beqs
d_{16}^{\text{(A)}}=-2.0 S(x_1+a\hat{\nu},t_1\mid x_3,t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}S(x_3,t_3\mid x_2,t_2)_{\substack{s_1\\c_0}\substack{s_2\\c_1}}S(x_0,t_0\mid x_1,t_1)_{\substack{s_7\\c_3}\substack{s_4\\c_3}}S(x_2+a\hat{\mu},t_2\mid x_0,t_0)_{\substack{s_3\\c_2}\substack{s_6\\c_3}}(1-\gamma_{\mu})_{s_2 s_3}(1-\gamma_{\nu})_{s_4 s_5}(U_{\mu}(x_2))_{{c_1c_2}}(U_{\nu}(x_1))_{{c_3c_4}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\\
d_{18}^{\text{(A)}}=2.0 S(x_1+a\hat{\nu},t_1\mid x_3,t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}S(x_3,t_3\mid x_2+a\hat{\mu},t_2)_{\substack{s_1\\c_0}\substack{s_2\\c_1}}S(x_0,t_0\mid x_1,t_1)_{\substack{s_7\\c_3}\substack{s_4\\c_3}}S(x_2,t_2\mid x_0,t_0)_{\substack{s_3\\c_2}\substack{s_6\\c_3}}(1+\gamma_{\mu})_{s_2 s_3}(1-\gamma_{\nu})_{s_4 s_5}(U_{\mu}^{\dagger}(x_2))_{{c_1c_2}}(U_{\nu}(x_1))_{{c_3c_4}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\\
d_{20}^{\text{(A)}}=2.0 S(x_1,t_1\mid x_3,t_3)_{\substack{s_5\\c_4}\substack{s_0\\c_0}}S(x_3,t_3\mid 

In [23]:
data={}
for k,v in topSorted.items():
    if v[0] in data.keys():
        data[v[0]].append(k)
    else:
        data[v[0]]=[k]
print(data)

{'A': [16, 18, 20, 22, 8, 10, 12, 14], 'B': [1, 3, 5, 7, 25, 31, 37, 43], 'C': [0, 2, 4, 6, 27, 33, 39, 45], 'D': [28, 34, 40, 46], 'E': [17, 19, 21, 23, 24, 30, 36, 42, 9, 11, 13, 15, 26, 32, 38, 44], 'F': [29, 35, 41, 47]}


In [27]:
def format_line(lst):
    s=""
    for i,c in enumerate(lst):
        if(i!=len(lst)-1):
            s+=c+" "
        else:
            s+=c
    return s

print("\\begin{tabular}{c c}")
for k,v in data.items():
    print("{} \& {}".format(k,format_line(v)))
print("\\end{tabular}")

\begin{tabular}{c c}


TypeError: unsupported operand type(s) for +: 'int' and 'str'